In [1]:
# Sensitivity analysis functions
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol

In [2]:
from __future__ import division
import numpy as np
from scipy.integrate import solve_ivp
from scipy import interpolate 
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from datetime import date
import datetime
from myfunctions_multi_scale import *
%matplotlib inline

In [3]:
# importing parameters for validation with Reading data 

xval = pd.ExcelFile('../data/input.xlsx')
dfval = xval.parse('Parameters_Reading',header=None)

# assigmment of parameters to values: 
for key,val in zip(dfval.iloc[:][0],dfval.iloc[:][1]):
    exec(key + '=val')
    print(key,val)

n_reactors = np.int(n_reactors)

umol_to_percent_DW 0.0014
n_days 14.0
Next0 110.0
Nint0 1.05
m0 0.19724999999999998
S 35.0
Nintcrit 2.0
n_reactors 1.0
Qp 459.5833333333333
Qsea 224910.0
Nsea 110.0
dilution 0.0
miu 0.027
Nintmax 4.2
Nintmin 0.7
KN 1.2
Ks 14.0
Vmax 60.0
Z 1.0
KI 20.0
K0 1.5
Ka 0.15
Topt 18.0
Tmin 5.0
Tmax 31.5
n 2.0
losses20 0.0016
teta 1.047
Sopt 18.0
Smin 0.0
Smax 45.0
Shigh 30.0
Slow 12.0


In [4]:
# Set of optimized parameters and ranges in which they are optimized


problem = {
    'num_vars': 7,
    'names': ['miu','KI','Ka', 'n','Tmax','Topt','losses20'],
    'bounds': [[0.025, 0.035],
               [15, 150],
               [0.01,0.2],
               [1, 6],
               [29, 32],
               [15, 20],
               [0.001,0.005]]
}
print(problem)

{'num_vars': 7, 'names': ['miu', 'KI', 'Ka', 'n', 'Tmax', 'Topt', 'losses20'], 'bounds': [[0.025, 0.035], [15, 150], [0.01, 0.2], [1, 6], [29, 32], [15, 20], [0.001, 0.005]]}


In [5]:
param_values = saltelli.sample(problem, 20)
print(param_values.shape)

(320, 7)


In [6]:
# import all data for model valodation (source: HOBO data from aerated cage in Reading - Alex Ch, 2017)

period1i = datetime.datetime(2017, 4, 20)
hour1i = 24 * (int(period1i.strftime("%j"))-1) + 8

period2i = datetime.datetime(2017, 5, 3)
hour2i = 24 * (int(period2i.strftime("%j"))-1) + 14

period3i = datetime.datetime(2017, 5, 17)
hour3i = 24 * (int(period3i.strftime("%j"))-1) + 14

period4i = datetime.datetime(2017, 6, 15)
hour4i = 24 * (int(period4i.strftime("%j"))-1) + 12

period5i = datetime.datetime(2017, 6, 28)
hour5i = 24 * (int(period5i.strftime("%j"))-1) + 11

x2 = pd.ExcelFile('../data/HOBO.xlsx')

dfV1 = x2.parse('period1_PAR',header=None)
dfV4 = x2.parse('period2_PAR',header=None)
dfV5 = x2.parse('period3_PAR',header=None)

#IV1 and TV1 based only on IMS and not on HOBO data

IV2, TV2,IV2_average,TV2_average = [],[],[],[]
for vali,valT in zip(dfV1.iloc[3:2488:1][5],dfV1.iloc[3:2488:1][3]):
    IV2.append(vali)
    TV2.append(float(valT))
for i in range(1,len(IV2),4):
    IV2_average.append(np.mean(IV2[i-1:i+3:1]))
    TV2_average.append(np.mean(TV2[i-1:i+3:1]))

IV3, TV3,IV3_average,TV3_average = [],[],[],[]
for vali,valT in zip(dfV1.iloc[3+14*24*4:2488:1][5],dfV1.iloc[3+14*24*4:2488:1][3]):
    IV3.append(vali)
    TV3.append(float(valT))
for i in range(1,len(IV3),4):
    IV3_average.append(np.mean(IV3[i-1:i+3:1]))
    TV3_average.append(np.mean(TV3[i-1:i+3:1]))
    
IV4, TV4,IV4_average,TV4_average = [],[],[],[]
for vali,valT in zip(dfV4.iloc[13:1249:1][5],dfV4.iloc[13:1249:1][3]):
    IV4.append(vali)
    TV4.append(float(valT))
for i in range(1,len(IV4),4):
    IV4_average.append(np.mean(IV4[i-1:i+3:1]))
    TV4_average.append(np.mean(TV4[i-1:i+3:1]))

IV5, TV5,IV5_average,TV5_average = [],[],[],[]
for vali,valT in zip(dfV5.iloc[7:1345:1][5],dfV5.iloc[7:1345:1][3]):
    IV5.append(vali)
    TV5.append(float(valT))
for i in range(1,len(IV5),4):
    IV5_average.append(np.mean(IV5[i-1:i+3:1]))
    TV5_average.append(np.mean(TV5[i-1:i+3:1]))

In [7]:
# Import of IMS I and T data
#ref: Chemodanov, A., Robin, A., Jinjikhashvily, G., Yitzhak, D., Liberzon, A., Israel, A., & Golberg, A. (2019).
# Feasibility study of Ulva sp.(Chlorophyta) intensive cultivation in a coastal area of the Eastern Mediterranean Sea.
# Biofuels, Bioproducts and Biorefining, 13(4), 864-877.
dFI = pd.read_csv('../data/ims_data_2017_PAR.csv',encoding= 'unicode_escape',header=None)
dfT = pd.read_csv('../data/ims_T_data_2017.csv',encoding= 'unicode_escape',header=None)

In [8]:
# process IMS I and T Data

# I data
days = list(range(1,366))
I = np.zeros(24*365)
Temp = np.zeros(24*365-1-24)
light_hours = list(range(5,19))
for day in days:
    for hour in light_hours:
        I[(day - 1)*24 + hour] = float(dFI.iloc[day][hour-1])

# T data
annual_hours_short = list(range(2,2741*3-1))
annual_hours = list(range(1,(366-1)*24+1))
f0 = interpolate.interp1d(annual_hours, I,kind = 'linear')

TV1 = []
for valT in zip(dfT.iloc[1:2741:1][4]):
    TV1.append(float(valT[0]))
TV1[-1]

TV_hours_reduced = annual_hours_short[0:-1:3]
f2 = interpolate.interp1d(TV_hours_reduced,TV1,kind = 'linear')
TV_interp = f2(annual_hours_short[0:-5:1])

In [9]:
# Lists of initial and boundary conditions and empiric results for returns 1-5:

t0_all = [hour1i, hour2i, hour3i, hour4i, hour5i]
tf_all = [hour1i + int(13*24), hour2i + int(14*24), hour3i + int(12*24-4), hour4i + int(13*24-4), hour5i + int(14*24-2)]
Nsea_all = [90, 111, 91, 148, 206] # in period 3 used final concentrations
mi_all = [1.315*0.15, 1.315*0.15, 1.315*0.15, 1.306*0.15, 1.306*0.15]
mf_real = [4590, 2781, 1120, 2001, 1069]
Nint0_all = [2, 1.05, 0.65, 3, 1.06]# Nint0 of first period - assumed 2 (after 1 week starvation), and of forth assumed 3 (after fertilizing in Porter)
Nintf_all = [1.05, 0.65, 0.98, 1.06, 1.40]
functions = [Reading_val, Reading_val_IMS]
TV_average = [TV1, TV2_average, TV3_average, TV4_average, TV5_average]
IV_average = [1, IV2_average, IV3_average, IV4_average, IV5_average]
xlabels_all = [['20 April', '27 April', '3 May'],['3 May', '10 May', '17 May'],['17 May', '23 May', '29 May'],['15 June', '22 june', '28 June'],['28 June', '5 July', '12 July']]

In [ ]:
# Calculating root mean square relative error (RMSRE)

Y = np.zeros([param_values.shape[0]])
evaluate_model = []
counter=0
for i, X in enumerate(param_values):
    miu = X[0]
    KI = X[1]
    Ka = X[2]   
    n = X[3]
    Tmax = X[4]
    Topt = X[5]
    losses20  = X[6]
    
    returns = [2, 4, 5]
    Errors = []
    for k in returns:
        t0 = t0_all[k-1]
        t = list(range(t0,tf_all[k-1]))
        Nsea = Nsea_all[k-1]
        Next0 = Nsea_all[k-1]
        Nint0 = Nint0_all[k-1]
        m0 = mi_all[k-1]
        TV_reduced = TV_average[k-1][1:len(t)+1]
        f1 = interpolate.interp1d(t, TV_reduced,kind = 'linear')    
        f0 = interpolate.interp1d(annual_hours, I,kind = 'linear')

        FINAL_M = []
        
        for j in range(2):
            function = functions[j]
            if j == 0:
                IV_reduced = IV_average[k-1][1:len(t)+1]
                f0 = interpolate.interp1d(t, IV_reduced,kind = 'linear')

            elif j == 1:
                f0 = interpolate.interp1d(annual_hours, I,kind = 'linear')

            NSEA, NEXT, NINT, M, Total_N, T, Total_biomass = [],[],[],[],[],[],[]

            # setup initial conditions identical for all reactors
            x0 = n_reactors*[Nsea, Next0, Nint0, m0]
            args = (Nintcrit,Nintmax,Nintmin,Vmax,Ks,KN,miu,S,Z,KI,K0,Ka,Topt,Tmin,Tmax,losses20,
                    teta,Sopt,Smin,Smax, Qp, Qsea, Nsea,f1,f0,dilution,n,umol_to_percent_DW)

            # solve the ODEs using the new syntax
            sol = solve_ivp(function, [t[0], t[-1]], x0, args=args, t_eval = t)

            # take the solution of the state variables:
            M_farm, N_farm = [], []
            NSEA.append(sol.y[0,:])
            NEXT.append(sol.y[1,:])
            NINT.append(sol.y[2,:])
            M.append(sol.y[3,:])
            M_farm.append((M[-1][-1] - m0) * 1.785)
            N_farm.append((M[-1][-1] * NINT[-1][-1] * 1.785 / 100))
            Total_biomass.append(round(sum(M_farm),3))
            Total_N.append(round(sum(N_farm),4))

            T.append(sol.t)
            t = T[0]

            final_m = round(M [-1][-1] * 1.785 / 0.15,4)
            measured_m = round(mf_real[k-1] * 1.785 / 1000,4)
            FINAL_M.append(final_m)

        error = abs(round((FINAL_M[-1] - FINAL_M[-2]) / FINAL_M[-1],3))
        square_error = error**2
        Errors.append(square_error)

    evaluate_model.append((np.mean(Errors))**0.5)
    Y[i] = evaluate_model[-1]
    print(counter)
    counter = counter+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [ ]:
print(np.median(Y))
print(np.std(Y))
print(np.mean(Y))
print(min(Y))
print(max(Y))

In [ ]:
Si = sobol.analyze(problem, Y,print_to_console=True)

In [ ]:
# Calculating relative errors and RMSRE

# model simulation

Ka = 0.15
KI = 20
n = 2
Tmax = 31.5
Topt = 18
losses20 = 0.0016

returns = [1, 2, 4, 5]
Errors = []
Square_errors = []

for k in returns:
    t0 = t0_all[k-1]
    t = list(range(t0,tf_all[k-1]))
    Nsea = Nsea_all[k-1]
    Next0 = Nsea_all[k-1]
    Nint0 = Nint0_all[k-1]
    m0 = mi_all[k-1]
    TV_reduced = TV_average[k-1][1:len(t)+1]
    f1 = interpolate.interp1d(t, TV_reduced,kind = 'linear')    
    f0 = interpolate.interp1d(annual_hours, I,kind = 'linear')

    NSEA, NEXT, NINT, M, Total_N, T, Total_biomass, FINAL_M = [],[],[],[],[],[],[],[]

    # setup initial conditions identical for all reactors
    x0 = n_reactors*[Nsea, Next0, Nint0, m0]
    args = (Nintcrit,Nintmax,Nintmin,Vmax,Ks,KN,miu,S,Z,KI,K0,Ka,Topt,Tmin,Tmax,losses20,
            teta,Sopt,Smin,Smax, Qp, Qsea, Nsea,f1,f0,dilution,n,umol_to_percent_DW)

    # solve the ODEs using the new syntax
    sol = solve_ivp(Reading_val_IMS, [t[0], t[-1]], x0, args=args, t_eval = t)

    # take the solution of the state variables:
    M_farm, N_farm = [], []
    NSEA.append(sol.y[0,:])
    NEXT.append(sol.y[1,:])
    NINT.append(sol.y[2,:])
    M.append(sol.y[3,:])
    M_farm.append((M[-1][-1] - m0) * 1.785)
    N_farm.append((M[-1][-1] * NINT[-1][-1] * 1.785 / 100))
    Total_biomass.append(round(sum(M_farm),3))
    Total_N.append(round(sum(N_farm),4))

    T.append(sol.t)
    t = T[0]

    final_m = round(M [-1][-1] * 1.785 / 0.15,4)
    measured_m = round(mf_real[k-1] * 1.785 / 1000,4)
    FINAL_M.append(final_m)
    error = abs(round((measured_m - final_m) / measured_m,4))
    square_error = error**2
    Errors.append(error)
    Square_errors.append(square_error)

    print('for return ' + str(k) + ' and using IMS irradiance data')
    print('The modeled m is: ' + str(final_m) + '[kg FW/cage] and the measured m is ' + str(measured_m) + '[kg FW/cage] \n')
    print('The calculated relative error is: ' + str(error) + '\n')

print('the mean error is: ' + str(np.mean(Errors)) + '\n')
print('the std  is: ' + str(np.std(Errors)) + '\n')
print('the RMSRE is: ' + str((np.mean(Square_errors))**0.5) + '\n')

In [ ]:
# 'miu','KI','Ka', 'n','Tmax','Topt','losses20'
Best_index = []
for i in range(len(Y)):
    if Y[i] < 1:
        Best_index.append(i)
        
b = param_values[Best_index].T

print(b)
fig,ax0 = plt.subplots(3,3,figsize=(14,10),sharey=True)
fig.delaxes(ax0[2,1]) #The indexing is zero-based here
fig.delaxes(ax0[2,2]) #The indexing is zero-based here

ax0.flat[0].plot(b[0],Y[Best_index],'.',markersize=4)
ax0.flat[0].set_xlabel('µmax\n [$hour^{-1}$]',fontsize=11, weight="bold")
ax0.flat[0].set_ylabel('RMSRE',fontsize=12, weight="bold")
ax0.flat[0].set_xlim([0.025, 0.035])

ax0.flat[1].plot(b[1],Y[Best_index],'.',markersize=4)
ax0.flat[1].set_xlabel('KI \n [µmol photons $s^{-1}$ $m^{-2}$]',fontsize=11, weight="bold")

ax0.flat[2].plot(b[2],Y[Best_index],'.',markersize=4)
ax0.flat[2].set_xlabel('Ka \n [$m^{2}$ $g DW^{-1}$]',fontsize=11, weight="bold")

ax0.flat[3].plot(b[3],Y[Best_index],'.',markersize=4)
ax0.flat[3].set_xlabel('n\n [-]',fontsize=11, weight="bold")
ax0.flat[3].set_ylabel('RMSRE',fontsize=12, weight="bold")

ax0.flat[4].plot(b[4],Y[Best_index],'.',markersize=4)
ax0.flat[4].set_xlabel('Tmax\n [C]',fontsize=11, weight="bold")

ax0.flat[5].plot(b[5],Y[Best_index],'.',markersize=4)
ax0.flat[5].set_xlabel('Topt\n [C]',fontsize=11, weight="bold")

ax0.flat[6].plot(b[6],Y[Best_index],'.',markersize=4)
ax0.flat[6].set_xlabel('λ\N{SUBSCRIPT TWO}\N{SUBSCRIPT ZERO}\n [$hour^{-1}$]',fontsize=11, weight="bold")
ax0.flat[6].set_xlim([0.0005, 0.0055])
ax0.flat[6].set_ylabel('RMSRE',fontsize=12, weight="bold")

plt.tight_layout()

plt.show()


name = 'fig_S7.png' 
fig.savefig(fname=name, dpi=600)